In [9]:
import pandas as pd

# Load the uploaded CSV file 
file_path = "C:/Users/Ashiya Garg/Downloads/Grocery_Inventory_and_Sales_Dataset.csv"
df = pd.read_csv(file_path)

# Step 1: Convert relevant date columns to datetime
date_columns = ["Date_Received", "Last_Order_Date", "Expiration_Date"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Step 2: Compute days_to_expiry
df["days_to_expiry"] = (df["Expiration_Date"] - df["Date_Received"]).dt.days

# Step 3: Clean Unit_Price (remove $, convert to float)
df["Unit_Price"] = df["Unit_Price"].replace('[\$,]', '', regex=True).astype(float)

# Step 4: Handle missing values
# Fill numerical columns with median
numerical_cols = ["Stock_Quantity", "Reorder_Level", "Reorder_Quantity",
                  "Unit_Price", "Sales_Volume", "Inventory_Turnover_Rate", "days_to_expiry"]
for col in numerical_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

# Fill categorical columns with mode
categorical_cols = ["Status", "Catagory", "Product_Name", "Warehouse_Location"]
for col in categorical_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode()[0])  
# Step 5: Encode categorical columns
df["Status_encoded"] = df["Status"].astype("category").cat.codes
df["Catagory_encoded"] = df["Catagory"].astype("category").cat.codes

# Step 6: Drop unnecessary columns for modeling
drop_cols = [
    "Product_ID", "Product_Name", "Supplier_ID", "Supplier_Name",
    "Warehouse_Location", "Date_Received", "Expiration_Date",
    "Last_Order_Date", "Status", "Catagory"
]
df_clean = df.drop(columns=drop_cols)
# Step 7: Remove expired entries
df_clean = df_clean[df_clean["days_to_expiry"] >= 0]

# Step 8: Outlier removal using IQR method
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[column] >= lower) & (df[column] <= upper)]

# Apply outlier removal on key numerical columns
outlier_columns = ["Unit_Price", "Sales_Volume", "Inventory_Turnover_Rate", "days_to_expiry"]
for col in outlier_columns:
    df_clean = remove_outliers_iqr(df_clean, col)

# Step 9: Final check (see if any missing values remain)
print(df_clean.isnull().sum())

# Optional: Preview cleaned data
df_clean.head()



Stock_Quantity             0
Reorder_Level              0
Reorder_Quantity           0
Unit_Price                 0
Sales_Volume               0
Inventory_Turnover_Rate    0
days_to_expiry             0
Status_encoded             0
Catagory_encoded           0
dtype: int64


,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Sales_Volume,Inventory_Turnover_Rate,days_to_expiry,Status_encoded,Catagory_encoded
0,22,72,70,4.5,32,19,34,2,4
2,30,38,83,6.0,31,34,50,1,4
4,37,30,74,4.0,62,25,94,1,3
6,96,52,16,2.5,67,13,94,0,5
7,44,90,17,2.5,21,91,2,2,2
